In [6]:
import numpy as np
import pandas as pd

In [52]:
# Simulation inventory system
# TODO: order rules.

class Simulation():
    """
        A simulation of a lost sales inventory management system. Replicates the inventory system of Zipkin (2008).
        Slight differences in that we asssume a time horizon of t=0,...,T-1 with terminal time T. In Zipkin (2008)
        the time horizon is t=1,...,T with terminal time T+1.
        
    ...

    Attributes
    ----------


    Methods
    -------

    """
    def __init__(self, T, lead_time, underage, overage, salvage, initial_inventory, order_rule, log_data=True):
        
        self.T = T
        self.L = lead_time # How long it takes for orders to arrive
        self.cu = underage # also known as per unit penalty cost
        self.co = overage # also known as per unit holding cost
        self.c = salvage  # also known as procurement
        self.init_inv = initial_inventory
        
        self.order_rule = order_rule# This should be an anonymous lambda function
        
        self.x = np.zeros((self.T+1, self.L+1))
        self.x[0] = self.init_inv
        self.log_data = log_data
        
        # If logging the data, create a dictionary to store updates
        if (self.log_data):
            self.log = {'StartingInv': [], 'Order': [], 'PostOrder': [], 'Demand': [],'PostDemand': [], 'PostDeliveryMovements': [],'PeriodCost': []}
        
    def order(self,t,x):
        """
            Here we define the various ordering policies
        """
        
        # Order up to, constant base-stock policy
        if (self.order_rule == 'CBS'):
            # Insert ordering policy here
            order_q = 5
        
        # Order-up-to, state-dependent base-stock policy
        if (self.order_rule == 'SDBS'):
            # Insert ordering policy here
            order_q = 5       
        
        # Fixed Order quantity, constant (i.e. Newsvendor)
        if (self.order_rule == 'FQ'):
            # Insert ordering policy here
            order_q = 5  
        
        # Order quantity, state dependent (i.e. Optimal)
        if (self.order_rule == 'OPT'):
            # Insert ordering policy here
            order_q = 5  
            
        return order_q
    
    def reset(self):
        """
            Reset the inventory simulation to run again
        """
        self.x = np.zeros((self.T+1, self.L+1))
        self.x[0] = self.init_inv
        self.period_cost = []
        if (self.log_data):
            self.log = {'StartingInv': [], 'Order': [], 'PostOrder': [], 'Demand': [],'PostDemand': [], 'PostDeliveryMovements': [],'PeriodCost': []}
        
        
        
    def run(self,demand):
        
        # Log period costs
        self.period_cost = []
        
        for t in range(self.T):
            
            if (self.log_data):
                self.log['StartingInv'].append(self.x[t].copy())
            
            # Step 1. Get stocking decision
            # could use an order-up-to policy OR fixed quantity. 
            # Pass the current inventory state if the former, as well as the time period
            Q = self.order(t, self.x[t][0])

            # Step 2. Add stocking decision to end of pipeline
            self.x[t][-1] = Q
            
            if (self.log_data):
                self.log['PostOrder'].append(self.x[t].copy())
                
            # Step 3. Demand in each channel is realised
            # Allow this to be negative for now so we know the lost sales penalty
            # But later we fix this to pass no negative inventory over.
            self.x[t][0] -= demand[t]
            
            # Step 4. Calculate period costs
            # if shortage: charge underage (lost sales penalty cost)
            # else if surplus: charge overage (holding cost)
            self.period_cost.append(np.abs(self.x[t][0]*self.cu) if self.x[t][0]<=0 else self.x[t][0]*self.co)
            
            # Step 5. Apply lost sales policy and carry move inventory through pipeline
            self.x[t+1][0] = max(self.x[t][0],0)
            for i in range(1,self.L+1):
                self.x[t+1][i-1]+=self.x[t][i]
                self.x[t+1][i] = 0
            
            if (self.log_data):
                self.log['Order'].append(Q)
                self.log['Demand'].append(demand[t])
                self.log['PostDemand'].append(self.x[t][0])
                self.log['PostDeliveryMovements'].append(self.x[t+1].copy())
                self.log['PeriodCost'].append(self.period_cost[t])
        
            
        # Salvage remaining inventory:
        # Here we will incur a overage cost from periods (T:T+L) for stock on-hand
        # and then salvage any remaining inventory at a unit cost c in time period T+L+1.
        # Common to just set this value to 0
        terminal_holding_costs = self.co*np.sum([self.x[self.T][i]*(self.L-i) for i in range(self.L)])
        self.period_cost.append(terminal_holding_costs-self.c*np.sum(self.x[self.T]))
        
        if (self.log_data):
            self.log['StartingInv'].append(self.x[self.T])
            self.log['PostOrder'].append(0)
            self.log['Order'].append(0)
            self.log['Demand'].append(0)
            self.log['PostDemand'].append(0)
            self.log['PostDeliveryMovements'].append(0)
            self.log['PeriodCost'].append(self.period_cost[self.T])
            
            # Export log to dataframe
            self.log = pd.DataFrame(self.log)
            self.log.index +=1
            self.log.index.name = 'Period'    
            
            

In [58]:
np.random.seed(37)
sim = Simulation(10,2,9,1,0,[5,5,5], 'FQ')
demand = np.random.poisson(5, 20)
sim.run(demand[:10])
print('Simulation 1: total cost: {}'.format(np.sum(sim.period_cost)))
display(sim.log)
sim.reset()
sim.run(demand[10:])
print('Simulation 2: total cost: {}'.format(np.sum(sim.period_cost)))
display(sim.log)

Simulation 1: total cost: 105.0


,StartingInv,Order,PostOrder,Demand,PostDemand,PostDeliveryMovements,PeriodCost
Period,,,,,,,
1,"[5.0, 5.0, 5.0]",5,"[5.0, 5.0, 5.0]",6,-1.0,"[5.0, 5.0, 0.0]",9.0
2,"[5.0, 5.0, 0.0]",5,"[5.0, 5.0, 5.0]",8,-3.0,"[5.0, 5.0, 0.0]",27.0
3,"[5.0, 5.0, 0.0]",5,"[5.0, 5.0, 5.0]",5,0.0,"[5.0, 5.0, 0.0]",0.0
4,"[5.0, 5.0, 0.0]",5,"[5.0, 5.0, 5.0]",5,0.0,"[5.0, 5.0, 0.0]",0.0
5,"[5.0, 5.0, 0.0]",5,"[5.0, 5.0, 5.0]",5,0.0,"[5.0, 5.0, 0.0]",0.0
6,"[5.0, 5.0, 0.0]",5,"[5.0, 5.0, 5.0]",3,2.0,"[7.0, 5.0, 0.0]",2.0
7,"[7.0, 5.0, 0.0]",5,"[7.0, 5.0, 5.0]",2,5.0,"[10.0, 5.0, 0.0]",5.0
8,"[10.0, 5.0, 0.0]",5,"[10.0, 5.0, 5.0]",8,2.0,"[7.0, 5.0, 0.0]",2.0
9,"[7.0, 5.0, 0.0]",5,"[7.0, 5.0, 5.0]",10,-3.0,"[5.0, 5.0, 0.0]",27.0


Simulation 2: total cost: 89.0


,StartingInv,Order,PostOrder,Demand,PostDemand,PostDeliveryMovements,PeriodCost
Period,,,,,,,
1,"[5.0, 5.0, 5.0]",5,"[5.0, 5.0, 5.0]",9,-4.0,"[5.0, 5.0, 0.0]",36.0
2,"[5.0, 5.0, 0.0]",5,"[5.0, 5.0, 5.0]",5,0.0,"[5.0, 5.0, 0.0]",0.0
3,"[5.0, 5.0, 0.0]",5,"[5.0, 5.0, 5.0]",4,1.0,"[6.0, 5.0, 0.0]",1.0
4,"[6.0, 5.0, 0.0]",5,"[6.0, 5.0, 5.0]",4,2.0,"[7.0, 5.0, 0.0]",2.0
5,"[7.0, 5.0, 0.0]",5,"[7.0, 5.0, 5.0]",6,1.0,"[6.0, 5.0, 0.0]",1.0
6,"[6.0, 5.0, 0.0]",5,"[6.0, 5.0, 5.0]",8,-2.0,"[5.0, 5.0, 0.0]",18.0
7,"[5.0, 5.0, 0.0]",5,"[5.0, 5.0, 5.0]",4,1.0,"[6.0, 5.0, 0.0]",1.0
8,"[6.0, 5.0, 0.0]",5,"[6.0, 5.0, 5.0]",3,3.0,"[8.0, 5.0, 0.0]",3.0
9,"[8.0, 5.0, 0.0]",5,"[8.0, 5.0, 5.0]",9,-1.0,"[5.0, 5.0, 0.0]",9.0
